In [1]:
import numpy as np
import random as r
from vpython import *

# global var later for pause button
is_paused = False

# set up my grid with each cell in my grid representing a voter
rows, cols = 50, 50
grid = np.zeros((rows, cols))
# 2d array to keep track of how many times a voter swings
swing_grid = np.zeros((rows, cols))
# create an empty grid to store VPython boxes, object type
box_grid = np.empty((rows, cols), dtype = object)
for i in range(rows):
    for j in range(cols):
        box_grid[i][j] = None
# creating my bg for the animation
scene.background = color.black
scene.center = vector(rows/2, cols/2 + 3, 0)
# title label
title = label(
    pos = vector(rows / 2, cols + 5, 0),
    text = "Simulating Voter Opinions",
    height = 16,
    box = True,
    color = color.white
)
# caption label (basically the key)
caption = label(
    pos = vector(rows / 2, cols + 1, 0),
    text = "Red = 0 | Blue = 1 | Yellow = Voter that swings the most",
    height = 10,
    box = False,
    color = color.white
)
# initializing my grid with random opinions
for i in range(rows):
    for j in range (cols): 
        # every voter has a randomly assigned opinion
        grid[i][j] = r.randint(0, 1)
        # creating boxes for every voter for my animation that's red or blue depending on opinion
        box_grid[i][j] = box(
            pos = vector(i, j, 0),
            size = vector(0.9, 0.9, 0.9), # so the boxes don't touch
            color = color.red if grid[i][j] == 0 else color.blue, # color depends on thier vote type
            opacity = 1.0
        )

# for a given voter, identify it's neighboring cells 
def find_neighbor(i, j, rows, cols):
    neighbors = []
    if i + 1 >= 0 and i + 1 < rows:
        neighbors.append((i + 1, j))

    if i - 1 >= 0 and i - 1 < rows:
        neighbors.append((i - 1, j))

    if j - 1 >= 0 and j - 1 < cols:
        neighbors.append((i, j - 1))

    if j + 1 >= 0 and j + 1 < cols:
        neighbors.append((i, j + 1))

    return neighbors

# this toggles if_paused on and off, currently set to false and changes the text of the button to "resume" if the is_paused is False
def pause():
    global is_paused, pause_button
    is_paused = not is_paused
    pause_button.text = "Resume" if is_paused == True else "Pause"

# this "resets" the animation so all the boxes are filled with random 0s or 1s
def reset():
    global red_count, blue_count
    red_count = 0
    blue_count = 0
    for i in range(rows):
        for j in range(cols):
            grid[i][j] = r.randint(0, 1)
            box_grid[i][j].color = color.red if grid[i][j] == 0 else color.blue
            box_grid[i][j].size = vector(0.9, 0.9, 0.9)

# this is the button that pauses the simulation
pause_button = button(
    bind = pause,
    text = "Pause" 
)

# this is the button that resets the animation
reset_button = button(
    bind = reset,
    text = "Reset"
)

# my buttons (will update them in my loop)
count_red = label(
    pos = vector(rows + 12, cols / 2 - 6, 0),
    text = "",
    height = 10,
    box = False,
    color = color.white
)
count_blue = label(
    pos = vector(rows + 12, cols / 2 + 6, 0),
    text = "",
    height = 10,
    box = False,
    color = color.white
)

# my step label
step_label = label(
    pos = vector(rows + 12, cols / 2, 0),
    text = "",
    height = 10,
    box = False,
    color = color.white
)

# overall loop logic
for num_steps in range(10000):
    rate(100)
    # runs only when it's not paused
    if is_paused == False:
        # creating a list for the positions of the neighbors of my random voter
        neighbor_positions = []
        # my random voters
        voter_i = r.randint(0, rows - 1)
        voter_j = r.randint(0, cols - 1)
        voter2_i = r.randint(0, rows - 1)
        voter2_j = r.randint(0, cols - 1)
        # i am also simulating random noise because sometimes voters can be irrational (there's a 5 percent chance a voter flips randomly)
        random_prob = 0.05
        if r.random() < random_prob:
            if grid[voter_i][voter_j] == 0:
                grid[voter_i][voter_j] = 1
                swing_grid[voter_i][voter_j] += 1
            else:
                grid[voter_i][voter_j] = 0
                swing_grid[voter_i][voter_j] += 1
        else:
            # find his neighbors positon
            neighbor_positions = find_neighbor(voter_i, voter_j, rows, cols)
            # finding the values of his neighbors
            neighbor_opinions = []
            for ni, nj in neighbor_positions:
                neighbor_opinions.append(grid[ni][nj])
            # if the neighbors have opinions give the voter a random opinion of a voter
            if neighbor_opinions:  
                grid[voter_i][voter_j] = r.choice(neighbor_opinions)
                swing_grid[voter_i][voter_j] += 1
        box_grid[voter_i][voter_j].color = color.red if grid[voter_i][voter_j] == 0 else color.blue

        # logic for introducing long range influences
        randi = (voter2_i + r.randint(0, rows - 1)) % rows # have random voter 2 influence a random voter & use mod to wrap around
        randj = (voter2_j + r.randint(0, cols - 1)) % cols
        if grid[voter2_i][voter2_j] == 0:
            grid[randi][randj] = 0
            swing_grid[randi][randj] += 1
            box_grid[randi][randj].color = color.red # changing the colors of the boxes
        else:
            grid[randi][randj] = 1
            swing_grid[randi][randj] += 1
            box_grid[randi][randj].color = color.blue
            
        # increase count by every 10 increase in num_steps to update the number of red & blue voters per 10 frames
        if num_steps % 10 == 0:
            red_count = 0
            blue_count = 0 # reset these to 0 every 10 steps and then recount the entire grid to update every 10 steps
            for i in range(rows):
                for j in range(cols):
                    if grid[i][j] == 0:
                        red_count += 1
                    else:
                        blue_count +=1
            # updating my buttons
            red_text = str(red_count)
            blue_text = str(blue_count)
            count_red.text = "Number of red voters: " + red_text
            count_blue.text = "Number of blue voters: " + blue_text
            step_label.text = "Number of steps: " + str(num_steps) # updating the label for number of steps
        
# creating a label at the end to signify that the simulation is over
end = label(
            pos = vector(rows / 2, cols / 2, 0),
            text = "End of Simulation!",
            height = 16,
            box = False,
            color = color.white
            )

# logic for introducing which voter swings the most
big_count = 0
coor_i = 0
coor_j = 0
for i in range(rows):
    for j in range(cols):
        if swing_grid[i][j] > big_count:
            big_count = swing_grid[i][j]
            coor_i = i
            coor_j = j
box_grid[coor_i][coor_j].color = color.yellow
box_grid[coor_i][coor_j].size = vector(5, 5, 5)

<IPython.core.display.Javascript object>

In [ ]:
import numpy as np
import random as r
from vpython import *

# set up my grid with each cell in my grid representing a voter
rows, cols = 50, 50
grid = np.zeros((rows, cols))
# 2d array to keep track of how many times a voter swings
swing_grid = np.zeros((rows, cols))
# create an empty grid to store VPython boxes, object type
box_grid = np.empty((rows, cols), dtype = object)
for i in range(rows):
    for j in range(cols):
        box_grid[i][j] = None
# creating my bg for the animation
scene.background = color.black
scene.center = vector(rows/2, cols/2 + 3, 0)

for i in range(10000):
    rate(100)
    for i in range(50):
        for j in range(50):
            box_grid[i][j] = box(
                pos = vector(i, j , 0),
                size = vector(1, 1, 0),
                color = r.choice((color.red, color.blue)), opacity = 1
            )